# LoyaltyOne

In [74]:
!ls

LoyaltyOne.ipynb  TRAINING.csv


In [75]:
!head TRAINING.csv

ID,Q201601_MILES,Q201601_SPONSORS,Q201602_MILES,Q201602_SPONSORS,Q201603_MILES,Q201603_SPONSORS,Q201604_MILES,Q201604_SPONSORS,FLAG,MILES201601GROCERY,MILES201602GROCERY,MILES201603GROCERY,MILES201604GROCERY,MILES201601CREDIT_CARD,MILES201602CREDIT_CARD,MILES201603CREDIT_CARD,MILES201604CREDIT_CARD,MILES201601OTHER,MILES201602OTHER,MILES201603OTHER,MILES201604OTHER,PROGRAM_TIER,PROVINCE,EMAIL_OPTIN,DEC_2016_ACCOUNT_BALANCE
1,58,1,36,2,38,2,72,2,0,58,35,37,56,0,0,0,16,0,1,1,0,B,ON,Y,266
2,43,4,153,4,186,3,246,4,1,7,22,12,23,29,126,174,219,7,5,0,4,B,?,N,28
4,291,8,455,4,711,7,338,8,0,33,1,41,50,135,342,465,123,123,112,205,165,G,AB,Y,6085
5,117,4,110,4,113,3,119,3,0,34,26,24,41,0,0,0,0,83,84,89,78,B,BC,Y,416
7,1222,4,2154,4,1405,4,1858,7,0,6,0,1,2,1201,2130,1367,1811,15,24,37,45,O,AB,N,13111
8,143,3,450,2,18,3,23,2,1,4,0,2,20,135,447,12,0,4,3,4,3,G,BC,Y,11666
9,282,3,303,2,224,2,268,3,0,12,26,15,28,269,277,209,239,1,0,0,1,G,QC,N,11345
11,56,1,186,1,224,2,26,1,0,0,0,0,0,56,186,222,26,0,0,2

In [76]:
import pandas, numpy as np
import matplotlib.pyplot as plt

In [77]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [78]:
trainData = pandas.read_csv('TRAINING.csv', na_values='?')
trainData = trainData.dropna(axis=1, how='any')

In [79]:
trainData

,ID,Q201601_MILES,Q201601_SPONSORS,Q201602_MILES,Q201602_SPONSORS,Q201603_MILES,Q201603_SPONSORS,Q201604_MILES,Q201604_SPONSORS,FLAG,...,MILES201601CREDIT_CARD,MILES201602CREDIT_CARD,MILES201603CREDIT_CARD,MILES201604CREDIT_CARD,MILES201601OTHER,MILES201602OTHER,MILES201603OTHER,MILES201604OTHER,EMAIL_OPTIN,DEC_2016_ACCOUNT_BALANCE
0,1,58,1,36,2,38,2,72,2,0,...,0,0,0,16,0,1,1,0,Y,266
1,2,43,4,153,4,186,3,246,4,1,...,29,126,174,219,7,5,0,4,N,28
2,4,291,8,455,4,711,7,338,8,0,...,135,342,465,123,123,112,205,165,Y,6085
3,5,117,4,110,4,113,3,119,3,0,...,0,0,0,0,83,84,89,78,Y,416
4,7,1222,4,2154,4,1405,4,1858,7,0,...,1201,2130,1367,1811,15,24,37,45,N,13111
5,8,143,3,450,2,18,3,23,2,1,...,135,447,12,0,4,3,4,3,Y,11666
6,9,282,3,303,2,224,2,268,3,0,...,269,277,209,239,1,0,0,1,N,11345
7,11,56,1,186,1,224,2,26,1,0,...,56,186,222,26,0,0,2,0,N,3362
8,12,1472,6,1625,5,1143,4,1232,5,0,...,1179,1044,1030,1071,93,69,71,63,Y,2313
9,13,58,2,48,4,50,4,94,6,0,...,17,0,16,30,41,43,34,63,Y,2350


In [80]:
trainData.iloc[1]

ID                            2
Q201601_MILES                43
Q201601_SPONSORS              4
Q201602_MILES               153
Q201602_SPONSORS              4
Q201603_MILES               186
Q201603_SPONSORS              3
Q201604_MILES               246
Q201604_SPONSORS              4
FLAG                          1
MILES201601GROCERY            7
MILES201602GROCERY           22
MILES201603GROCERY           12
MILES201604GROCERY           23
MILES201601CREDIT_CARD       29
MILES201602CREDIT_CARD      126
MILES201603CREDIT_CARD      174
MILES201604CREDIT_CARD      219
MILES201601OTHER              7
MILES201602OTHER              5
MILES201603OTHER              0
MILES201604OTHER              4
EMAIL_OPTIN                   N
DEC_2016_ACCOUNT_BALANCE     28
Name: 1, dtype: object

#### Split X and Y

In [81]:
Y = trainData.FLAG
X = trainData.drop('FLAG', axis=1)
X.EMAIL_OPTIN = X.EMAIL_OPTIN.map(dict(Y=1, N=0))
X = X.as_matrix().astype(float)

In [115]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
#encoded_Y = np_utils.to_categorical(encoded_Y)

In [116]:
encoded_Y

array([0, 1, 0, ..., 0, 1, 0])

In [120]:
# baseline model
model = Sequential()
def create_baseline():
    # create model
    
    #model.add(keras.layers.normalization.BatchNormalization(axis=0))
    model.add(Dense(30, input_dim=23, init="normal", activation='relu', W_regularizer=keras.regularizers.l2(0.01)))
    #model.add(Dense(20, init="normal", activation='relu', W_regularizer=keras.regularizers.l2(0.01)))
    model.add(Dense(1, activation='hard_sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [121]:
estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=10, batch_size=80, verbose=0)
kfold = StratifiedKFold(n_splits=3, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("\n Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock



 Results: 77.62% (0.00%)


In [123]:
preds = model.predict(X[:100])

In [126]:
preds

array([[ 0.22821037],
       [ 0.2282104 ],
       [ 0.22821021],
       [ 0.22821037],
       [ 0.22821002],
       [ 0.22821005],
       [ 0.22821005],
       [ 0.22821029],
       [ 0.2282103 ],
       [ 0.2282103 ],
       [ 0.22821005],
       [ 0.2282104 ],
       [ 0.22820993],
       [ 0.2282103 ],
       [ 0.22821021],
       [ 0.2282103 ],
       [ 0.22821029],
       [ 0.2282104 ],
       [ 0.22821023],
       [ 0.22820985],
       [ 0.22821005],
       [ 0.22821021],
       [ 0.22821033],
       [ 0.22820994],
       [ 0.2282103 ],
       [ 0.22820985],
       [ 0.22821026],
       [ 0.22820994],
       [ 0.22820985],
       [ 0.2282103 ],
       [ 0.22821034],
       [ 0.22821021],
       [ 0.22821009],
       [ 0.22821034],
       [ 0.22820985],
       [ 0.22821037],
       [ 0.2282103 ],
       [ 0.2282103 ],
       [ 0.22821018],
       [ 0.22821037],
       [ 0.2282103 ],
       [ 0.22821026],
       [ 0.2282103 ],
       [ 0.22820985],
       [ 0.2282103 ],
       [ 0

In [124]:
mean = preds.sum()/len(preds)

In [125]:
mean

0.22821020126342773

In [127]:
bins = []
for v in preds:
    if v < mean:
        bins.append(0)
    else:
        bins.append(1)

In [128]:
bins

[1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1]